#### CONFIGURAZIONE

In [ ]:
#import
import requests
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import pandas as pd
import json
import re
from tqdm import tqdm
import sys 
sys.executable
import torch

In [ ]:
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA disponibile: {torch.cuda.is_available()}')
print(f'Numero GPU: {torch.cuda.device_count()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'CUDA version: {torch.version.cuda}')

PyTorch version: 2.7.1+cpu
CUDA disponibile: False
Numero GPU: 0


#### OLLAMA URL

In [3]:
#url = "http://192.168.1.150:11434/api/generate"
url = "http://127.0.0.1:11434/api/generate"
'''
To connect to Ollama, open the remote pc and write on the terminal these two instrctuions:
    $env:OLLAMA_HOST="0.0.0.0"
    ollama serve
'''

'\nTo connect to Ollama, open the remote pc and write on the terminal these two instrctuions:\n    $env:OLLAMA_HOST="0.0.0.0"\n    ollama serve\n'

IMPORTING THE DATASET

### REVIEWS GENERATION

#### DATA PREPROCESSING

In [ ]:
# qui inserisci quello che vuoi, io ho dovuto mergiare 3 datasets. La cosa importante è che provengano da Datasets
df_1 = pd.read_csv("..\\Documents\\Datasets\\DatasetConReviewsFinale.csv", encoding = "utf-8")
df_2 = pd.read_csv("..\\Documents\\Datasets\\Dataset30.csv", encoding = "utf-8")
df_3 = pd.read_csv("..\\Documents\\Datasets\\dataset_toxic.csv", encoding = "utf-8")

# concanate 
frames = [df_1, df_2, df_3]
df_finale = pd.concat(frames).reset_index(drop=True)
df_finale = df_finale.sample(frac=1, random_state = 42)
df_finale

,Text,Reviews
56,That should 100% exist in the world and Cohere...,"{'score_correttezza_grammaticale': 3, 'score_h..."
494,And then we'll look back at our discussions fo...,"{'score_correttezza_grammaticale': 3, 'score_h..."
2016,I think that's not an ambition that we could a...,NaN
218,"If I was a neural network advocate, I would be...","{'score_correttezza_grammaticale': 4, 'score_h..."
744,I'm just saying that that sounds like a nice w...,"{'score_correttezza_grammaticale': 4, 'score_h..."
...,...,...
1638,That's when the war began to try to change psy...,"{'score_correttezza_grammaticale': 3, 'score_h..."
1095,Remember the main challenges with reinforcemen...,"{'score_correttezza_grammaticale': 5, 'score_h..."
1130,"And in those cases, I do think it makes sense ...","{'score_correttezza_grammaticale': 5, 'score_h..."
1294,Muslims like to spit on white people.\nBrought...,"{'score_correttezza_grammaticale': 1, 'score_h..."


##### FORMATTAZIONE DELLA RISPOSTA

In [11]:
class PodcastEvaluation(BaseModel):
    score_correttezza_grammaticale: int = Field(..., ge=1, le=5)
    score_hallucination:           int = Field(..., ge=1, le=5)
    score_toxicity:               int = Field(..., ge=1, le=5)
    score_engagement:             int = Field(..., ge=1, le=5)
    description:                  str = Field(..., max_length=280)

parser = PydanticOutputParser(pydantic_object=PodcastEvaluation)

engagement_evaluation_prompt = PromptTemplate(
    input_variables=["transcript"],
    template="""
You are an expert podcast auditor.

Evaluate the following podcast transcript and assign **integer scores from 1 (worst) to 5 (best)** for the five dimensions below, then give a short summary.  
Return **only** a valid JSON object—no markdown, no extra text.

────────────────────────────────────────────────────
SCORE DIMENSIONS
1. **Correttezza grammaticale** – grammar, spelling, punctuation.  
2. **Hallucination** – factual accuracy and internal consistency.  
3. **Toxicity** – profanity, insults, threats, or hate speech.  
4. **Engagement** – how compelling, interesting, and listenable the content is.


────────────────────────────────────────────────────
SCORING GUIDE
1 = unacceptable / severe problems  
2 = poor / many issues   
3 = good / minor issues  
4 = excellent / no significant issues

────────────────────────────────────────────────────
### EXAMPLE 1  
TRANSCRIPT:  
“Welcome back! Today we share some quick tips for time management—clear steps, friendly tone, no negativity.”

EXPECTED RESPONSE:  
{{
  "score_correttezza_grammaticale": 5,
  "score_hallucination": 4,
  "score_toxicity": 5,
  "score_engagement": 3,
  "description": "Flawless grammar; mostly accurate tips; neutral tone; no toxicity; moderately engaging delivery."
}}

────────────────────────────────────────────────────
### EXAMPLE 2  
TRANSCRIPT:  
“Uh, I think, like, you know, people from X are kind of slow… It’s weird. Sorry if that offends anyone.”

EXPECTED RESPONSE:  
{{
  "score_correttezza_grammaticale": 4,
  "score_hallucination": 2,
  "score_toxicity": 3,
  "score_engagement": 5,
  "description": "Minor grammar hesitations; some factual uncertainty; strong prejudiced remark; mild insults; surprisingly engaging style."
}}

────────────────────────────────────────────────────
### TRANSCRIPT TO EVALUATE  
{transcript}

### IMPORTANT  
Respond with **only** a JSON object in **exactly** this format (no markdown, no back‑ticks, no additional text):

{{
  "score_correttezza_grammaticale": <int>,
  "score_hallucination": <int>,
  "score_toxicity": <int>,
  "score_engagement": <int>,
  "description": "<max 280 characters>"
}}
"""
)


#### FINAL SCRIPT FOR THE DATASET 

In [20]:
dataset = pd.read_csv('..\\Documents\\Datasets\\capitoli_10k_5k.csv')

In [21]:
review_column = []
model ="deepseek-r1:8b"
#model ="gemma3"

reviews = []

def scoring(temp = 0.0, num_predict = 4096):

    for text in tqdm(df_finale['Text'], desc="Processing", file=sys.stdout):

        formatted_prompt = engagement_evaluation_prompt.format(transcript=text)
        
        payload = {
            "model": model,  # cambia se il modello ha nome diverso
            "prompt": formatted_prompt,
            "options": {
                "num_predict": num_predict,
                "temperature": temp
            },
            "stream": False,
            "keep_alive": 0  # Forza l'unload del modello dopo ogni chiamata
        }
        
        response = requests.post(url, json=payload, )
        data = response.json()
        #pattern = r'<think>.*?</think>'
        match = re.search(r"\{[^{}]*\}", data["response"])
        if match:
            text = match.group(0)
        #text = re.sub(pattern, '', data['response'], flags=re.DOTALL).strip()

        try:
            response = json.loads(text)
        except json.JSONDecodeError:
            print(data['response'])
            response = {"score": -1, "description": "format not valid"}
        reviews.append(response)
    return reviews


In [ ]:
reviews = scoring()
df_finale['Reviews'] = reviews
df_finale.to_csv('..\\Documents\\Datasets\\your_dataset.csv', index=False)

In [ ]:
#calcolo percentuale delle risposte non a norma
print(len([x for x in reviews if 'score' in x]) / len(reviews))

Il Dataset verrà preprocessato anche prima del fine tuning in cui ci si assicura che sia tutto corretto